# Setup

Initial module setup.

In [1]:
import numpy.typing as np_types 

from xgboost import XGBClassifier
from eeg_auth_models_framework import data, pre_process, features, training, model, processor
from eeg_auth_models_framework.utils import conversion

# Constants

In [2]:
DATASET_SAMPLE_FREQ_HZ = 200
DATA_CHANNEL_NAMES = ['T7','F8','Cz','P4']
FREQUENCIES = [
    pre_process.FrequencyBand(lower=8.0, upper=12.0, label='Alpha'),
    pre_process.FrequencyBand(lower=12.0, upper=35.0, label='Beta'),
    pre_process.FrequencyBand(lower=4.0, upper=8.0, label='Theta'),
    pre_process.FrequencyBand(lower=35.0, upper=None, label='Gamma'),
    pre_process.FrequencyBand(lower=None, upper=None, label='Raw'),
]
WINDOW_SIZE = 1200
WINDOW_OVERLAP = 0.5
K_FOLDS = 10

# Model Builder Configuration

Configure data source, data reading method, data labelling method, and training process. 

In [3]:
class XGBBuilder(model.ModelBuilder[XGBClassifier]):    
    def create_classifier(self) -> XGBClassifier:
        # see: https://github.com/Rahul2K20/EEG_Authentication/blob/main/Main.ipynb
        return XGBClassifier(
            n_estimators=100,
            max_depth=3,
            learning_rate=0.1
        )
    
    def train_classifier(self, classifier: XGBClassifier, x_data: np_types.ArrayLike, y_data: np_types.ArrayLike):
        classifier.fit(x_data, y_data)
        
    def score_classifier(self, classifier: XGBClassifier, x_data: np_types.ArrayLike, y_data: np_types.ArrayLike) -> float:
        return classifier.score(x_data, y_data)

# Configuration

In [4]:
downloader = data.AuditoryDataDownloader()
reader = data.AuditoryDataReader()
labeller = training.SubjectDataLabeller()
converter_filter = conversion.MNEDataFrameConverter(
    channels=DATA_CHANNEL_NAMES, 
    sample_frequency=DATASET_SAMPLE_FREQ_HZ
)
converter_psd = conversion.MNEDataFrameConverter(sample_frequency=DATASET_SAMPLE_FREQ_HZ)

# Data Processing

## Pre-Processing Steps

Define pre-processing steps to be used in model.

In [5]:
pre_process_steps = pre_process.PreProcessingPipeline([
    pre_process.EEGBandpassFilterStep(
        FREQUENCIES, 
        converter_filter
    ),
    pre_process.DataWindowStep(WINDOW_SIZE, WINDOW_OVERLAP)
])

## Feature Extraction Steps

Define feature extraction steps to be applied to the pre-processed data.

In [6]:
feature_extraction_steps = features.FeatureExtractPipeline([
    features.PSDExtractor(converter_psd)
])

## Data Processor

In [7]:
data_processor = processor.DataProcessor(
    pre_process=pre_process_steps,
    feature_extraction=feature_extraction_steps
)

# Training

Execute training of authentication models.

In [8]:
xgb_builder = XGBBuilder(
    data_downloader=downloader,
    data_reader=reader,
    data_labeller=labeller,
    data_processor=data_processor
)
results = xgb_builder.train(K_FOLDS)
print('Average scores and training times:')
for subject, average in results.iter_subject_average_scores():
    training_time = results.training_statistics[subject].training_duration
    print(f'{subject}: {average} ({training_time} seconds)')
print(f'Global average score: {results.global_average_score}')
print(f'Global average training time: {results.global_average_time} seconds')

Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
    Range : 0 ... 1199 =      0.000 ...     5.995 secs
Ready.
Effective window size : 6.000 (s)
Creating RawArray with float64 data, n_channels=20, n_times=1200
  